In [1]:
#Donald Dunagan
#811-648-053
#29 September 2018
#CSCI 6560 Term Project
#The use of evolutionary computation in the
#optimization of neural networks

# Set Up Environment

In [1]:
#imports

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.utils import shuffle
from sklearn.metrics import make_scorer, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV
from scipy import stats
import random, operator, time, itertools, math, numpy
from deap import algorithms, base, creator, tools
import warnings

In [2]:
#import Parkinson's Dataset and visualize

PDDF =  pd.read_csv("C:\\Users\\gray\\Desktop\\Evolutionary_Computing\\Term_Project\\Parkinson's_Data.csv")
print ('The shape of the dataframe is:',PDDF.shape)
PDDF.head()

The shape of the dataframe is: (5875, 22)


,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


#### Initial shuffling and splitting

In [3]:
#shuffle the data
PDDF = shuffle(PDDF)

#isolate features
features_to_drop = ['subject#','age','sex','test_time','motor_UPDRS','total_UPDRS']
just_features = PDDF.drop(features_to_drop,axis=1)
    
#isolate motor_UPDRS
just_motor = PDDF.motor_UPDRS
   
#isolate total_UPDRS
just_total = PDDF.total_UPDRS

#split into training and testing data
training_features = just_features[:5287]
testing_features = just_features[5287:]
    
training_motor = just_motor[:5287]
testing_motor = just_motor[5287:]
    
training_total = just_total[:5287]
testing_total = just_total[5287:] 

#### A function for shuffling and splitting the data

In [4]:
def shuffleSplit():
    '''
    Shuffles and splits the DataFrame     
    '''
    global PDDF
    
    #shuffle the data
    PDDF = shuffle(PDDF)

    #isolate features
    features_to_drop = ['subject#','age','sex','test_time','motor_UPDRS','total_UPDRS']
    global just_features
    just_features = PDDF.drop(features_to_drop,axis=1)
    
    #isolate motor_UPDRS
    global just_motor
    just_motor = PDDF.motor_UPDRS
   
    #isolate total_UPDRS
    global just_total
    just_total = PDDF.total_UPDRS

    #split into training and testing data
    global training_features
    training_features = just_features[:5287]
    global testing_features
    testing_features = just_features[5287:]
    
    global training_motor
    training_motor = just_motor[:5287]
    global testing_motor
    testing_motor = just_motor[5287:]
    
    global training_total
    training_total = just_total[:5287]
    global testing_total
    testing_total = just_total[5287:]  
    
    return

In [5]:
#cross validation needs to return the mean absolute error (MAE) for each fold
#so i have to make a custom scoring object

#create scorer object
myScorer = make_scorer(mean_absolute_error)

In [6]:
PDDF.head()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
4215,31,75,0,76.348,30.895,33.674,0.01091,0.000075,0.00531,0.00621,...,1.011,0.05176,0.07440,0.09892,0.15529,0.056160,15.339,0.70005,0.77993,0.345170
618,5,75,0,14.376,31.000,39.469,0.00289,0.000026,0.00124,0.00166,...,0.245,0.01664,0.01923,0.02332,0.04993,0.008739,23.168,0.58170,0.65362,0.143280
2073,15,65,0,117.520,15.000,20.667,0.00247,0.000014,0.00123,0.00108,...,0.185,0.00977,0.01005,0.01265,0.02930,0.004328,29.212,0.40086,0.60610,0.092811
3161,23,59,1,51.421,13.574,24.198,0.00403,0.000021,0.00202,0.00221,...,0.187,0.01094,0.01305,0.02010,0.03281,0.009401,22.335,0.51208,0.64130,0.213230
5660,41,68,1,119.490,37.297,45.438,0.00864,0.000046,0.00435,0.00449,...,0.270,0.01177,0.01267,0.01604,0.03531,0.045779,20.813,0.65126,0.67992,0.370020


# Scikit Learn MLP

## Baselines for untuned, default MLP regressors

### 10-fold cross validated training MAE of an untuned MLP regressor targeting motor_UPDRS

In [36]:
#10-fold cross validated untuned MLP targeting motor_UPDRS

MLP = MLPRegressor()

#10-fold CV
vals = cross_val_score(MLP,training_features,training_motor,cv=10, scoring=myScorer)
print("MAE: {}".format(vals.mean()))

MAE: 6.918117587682678


### 10-fold cross validated training MAE of an untuned MLP regressor targeting total_UPDRS

In [37]:
#10-fold cross validated untuned MLP targeting total_UPDRS

MLP = MLPRegressor()

#10-fold CV
vals = cross_val_score(MLP,training_features,training_total,cv=10, scoring=myScorer)
print("MAE: {}".format(vals.mean()))

MAE: 8.66401242526413


### Playing with hidden layers

#### "Rule of thumb": taking average of input size + output size = 8

In [31]:
#10-fold cross validated MLP targetting total_UPDRS

warnings.filterwarnings("ignore")
MLP = MLPRegressor(hidden_layer_sizes=(8,))

#10-fold CV
vals = cross_val_score(MLP,training_features,training_total,cv=5, scoring=myScorer)
print("MAE: {}".format(vals.mean()))

MAE: 8.72974978185929


## Default parameters of MLP

In [26]:
MLP = MLPRegressor()
MLP.get_params

<bound method BaseEstimator.get_params of MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)>

In [31]:
print('One hidden layer of size 100')
print('Widght optimization solver is adam')
print('Beta1 = 0.9')
print('Beta2 = 0.999')
print('Training is not stopped early')
print('adam epsilon is 1e-08')
print('The learning rate is constant')
print('The learning rate is 0.001')
print('The maximum number of epochs is 200')

One hidden layer of size 100
Widght optimization solver is adam
Beta1 = 0.9
Beta2 = 0.999
Training is not stopped early
adam epsilon is 1e-08
The learning rate is constant
The learning rate is 0.001
The maximum number of epochs is 200


## Genetic algorithms stuff

#### Evaluation Function

In [11]:
def evalNN(features, actual, individual):
    """
    Takes the actual UPDRS values (either motor or total) and an np 
    array of length 145, extracts the weights for layer 2,
    the biases for layer 2, the weights for layer 3, and the bias for
    layer 3, runs the neural network, and calculates the mean absolute 
    error between predicted UPDRS and actual UPDRS
    """
    #extract matrices
    weights2 = individual[0:128].reshape(16,8)
    bias2 = individual[128:136].reshape(8)
    
    weights3= individual[136:144].reshape(8,1)
    bias3 = individual[144]
    
    
    #perform matrix operations that take place inside the 
    #neural network
    
    #matrix multiplication between feature matrix and weights of hidden layer
    hidden = np.matmul(features,weights2)
    #add biases of hidden layer to this
    hidden = hidden + bias2
    
    #perform ReLu activation function
    np.maximum(hidden,0,hidden)
    
    
    #matrix multiplication between the result of the hidden layer and 
    #the weights of the outpul layer
    final = np.matmul(hidden,weights3)
    #add bias of output layer to this
    final = final + bias3
    #activation function for output node of regression is linear
    #so this is result of the neural network
    
    #calculate mean absolute error between predicted and actual and return
    print (len(final))
    
    return mean_absolute_error(actual,final)

In [14]:
this = np.random.rand(145)
evalNN(training_features,training_total,this)

5287


13.851140927995356

#### Creator

In [43]:
#python throws a warning due to an intentional duplicate class

warnings.filterwarnings('ignore')

#create a fitness is to be minimized
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#define an individual's representation as list
creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
for i in range(1,15):
    print (i,":",(evalNN(i)))

C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


1 : 8.818103214026122


C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

2 : 8.965534527289812


C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


6 : 8.658913513449109
7 : 8.611352658825183


C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


8 : 8.718596172343812
9 : 8.607337345998719


C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


#### Toolbox

In [ ]:
toolbox = base.Toolbox()

#candidate solutions are permutations
toolbox.register("candidate", random.sample, range(numQueens), numQueens)

#set structure for individuals
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#set evalutation function
toolbox.register("evaluate", evalNN)
#crossover is partial crossover
toolbox.register("mate", tools.cxPartialyMatched)
#mutation is index shuffling
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=2.0/numQueens)
#uses tournament selecton
toolbox.register("select", tools.selTournament, tournsize=3)

#### Create the Initial Population and Run the Algorithm

In [ ]:
#generate the intial population
pop = toolbox.population(n=180)
#run the algorithm
result, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10000, verbose=False)

#### Results

In [ ]:
print("The solution: {}".format(tools.selBest(result,k=1)[0]))
print("Number of conflicts: {}".format(evalNQueens(tools.selBest(result,k=1)[0])))